In [1]:
using Laplacians

objc[44522]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[44522]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[44522]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[44522]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
set_bigfloat_precision(1024)

1024

In [34]:
n = 1250;
a = chimera(n,44); la = lap(a);
x = rand(n);
b = la * x; b = b - mean(b);

In [35]:
srand(1)
tree = lap(akpw(a)); 
cf = cholfact(tree,shift=1e-15);
p = cf[:p]
invp = invperm(p)
L = sparse(cf[:L])
U = L';

In [36]:
F = function(b)
    return cf \ (b - mean(b))
end

(anonymous function)

In [37]:
@time f = pcg(la, b, F, tol=1e-1, verbose=true);

PCG stopped after: 28 iterations with relative error 0.08917886584058253.
  0.006768 seconds (919 allocations: 2.979 MB)


In [38]:
function to1024{Tv,Ti}(mat::SparseMatrixCSC{Tv,Ti})
    u,v,w = findnz(mat)
    w1024 = Array{BigFloat,1}(0)
    for val in w
        push!(w1024, BigFloat(val))
    end
    return sparse(u,v,w1024)
end

function to1024{Tv}(v::Array{Tv,1})
    res = Array{BigFloat,1}(0)
    for val in v
        push!(res, BigFloat(val))
    end
    return res
end

to1024 (generic function with 2 methods)

In [39]:
la1024 = to1024(la)
U1024 = UpperTriangular(to1024(U))
b1024 = to1024(b);

In [40]:
F1024 = function(b::Array{BigFloat,1})
    res = (copy(b) - mean(b))[p]

    res = U1024' \ res
    res = U1024 \ res

    return res[invp]
end

(anonymous function)

In [41]:
@time f = pcg(la1024, b1024, F1024, tol=1e-1, verbose=true);

PCG stopped after: 25 iterations with relative error 9.269389465694072485985781903153963402614951052108239043860994925138819148495447332827251563655621499317190359215691082440522056765280922915835569081871240504540613254748629253398530324824666742509384580906655045361732608146185248975495378433839614700876854422087438410906868465751089805144162611329514596869406e-02.
 35.821287 seconds (178.37 M allocations: 11.078 GB, 43.02% gc time)
